<img src="https://ucfai.org/core/sp20/time-series/banner.png">

<div class="col-12">
    <h1> Time Series Analysis </h1>
    <hr>
</div>

<div style="line-height: 2em;">
    <p>by: 
        <a href="https://ucfai.org/authors/nspeer12">@nspeer12</a>
        
         on Mar 04, 2020</p>
</div>

In [ ]:
from pathlib import Path

DATA_DIR = Path("/kaggle/input")
if (DATA_DIR / "ucfai-core-sp20-time-series").exists():
    DATA_DIR /= "ucfai-core-sp20-time-series"
else:
    # You'll need to download the data from Kaggle and place it in the `data/`
    #   directory beside this notebook.
    # The data should be here: https://kaggle.com/c/ucfai-core-sp20-time-series/data
    DATA_DIR = Path("data")